In [1]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io
import requests
import re

In [2]:
r = requests.get("https://data.heatonresearch.com/data/t81-558/text/treasure_island.txt")
raw_text = r.text
print(raw_text[0:1000])
print(len(raw_text))

ï»¿The Project Gutenberg EBook of Treasure Island, by Robert Louis Stevenson

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: Treasure Island

Author: Robert Louis Stevenson

Illustrator: Milo Winter

Release Date: January 12, 2009 [EBook #27780]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK TREASURE ISLAND ***




Produced by Juliet Sutherland, Stephen Blundell and the
Online Distributed Proofreading Team at http://www.pgdp.net









 THE ILLUSTRATED CHILDREN'S LIBRARY


         _Treasure Island_

       Robert Louis Stevenson

          _Illustrated by_
            Milo Winter


           [Illustration]


           GRAMERCY BOOKS
              NEW YORK




 Foreword copyright Â© 1986 by Random House V
397419


In [0]:
processed_text = raw_text.lower()
processed_text = re.sub(r'[^\x00-\x7f]',r'', processed_text)

In [4]:
print('corpus length:', len(processed_text))

chars = sorted(list(set(processed_text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 397400
total chars: 60


In [0]:
seq_length = 100
x_data = []
y_data = []

In [0]:
for i in range(0, len(processed_text) - seq_length, 3):
    # Define input and output sequences
    # Input is the current character plus desired sequence length
    in_seq = processed_text[i:i + seq_length]

    # Out sequence is the initial character plus total sequence length
    out_seq = processed_text[i + seq_length]

    # We now convert list of characters to integers based on
    # previously and add the values to our lists
    x_data.append([char_indices[char] for char in in_seq])
    y_data.append(char_indices[out_seq])

In [7]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 132434


In [8]:
len(y_data)

132434

In [9]:
print(x_data[:10])

[[53, 41, 38, 2, 49, 51, 48, 43, 38, 36, 53, 2, 40, 54, 53, 38, 47, 35, 38, 51, 40, 2, 38, 35, 48, 48, 44, 2, 48, 39, 2, 53, 51, 38, 34, 52, 54, 51, 38, 2, 42, 52, 45, 34, 47, 37, 13, 2, 35, 58, 2, 51, 48, 35, 38, 51, 53, 2, 45, 48, 54, 42, 52, 2, 52, 53, 38, 55, 38, 47, 52, 48, 47, 1, 0, 1, 0, 53, 41, 42, 52, 2, 38, 35, 48, 48, 44, 2, 42, 52, 2, 39, 48, 51, 2, 53, 41, 38, 2, 54], [2, 49, 51, 48, 43, 38, 36, 53, 2, 40, 54, 53, 38, 47, 35, 38, 51, 40, 2, 38, 35, 48, 48, 44, 2, 48, 39, 2, 53, 51, 38, 34, 52, 54, 51, 38, 2, 42, 52, 45, 34, 47, 37, 13, 2, 35, 58, 2, 51, 48, 35, 38, 51, 53, 2, 45, 48, 54, 42, 52, 2, 52, 53, 38, 55, 38, 47, 52, 48, 47, 1, 0, 1, 0, 53, 41, 42, 52, 2, 38, 35, 48, 48, 44, 2, 42, 52, 2, 39, 48, 51, 2, 53, 41, 38, 2, 54, 52, 38, 2], [48, 43, 38, 36, 53, 2, 40, 54, 53, 38, 47, 35, 38, 51, 40, 2, 38, 35, 48, 48, 44, 2, 48, 39, 2, 53, 51, 38, 34, 52, 54, 51, 38, 2, 42, 52, 45, 34, 47, 37, 13, 2, 35, 58, 2, 51, 48, 35, 38, 51, 53, 2, 45, 48, 54, 42, 52, 2, 52, 53, 38

In [10]:
len(x_data[0])

100

In [0]:
import numpy as np

X = np.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(len(chars))

In [12]:
X.shape

(132434, 100, 1)

In [13]:
from keras.utils import np_utils

y = np_utils.to_categorical(y_data)

Using TensorFlow backend.


In [14]:
y.shape

(132434, 60)

In [15]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 256)          264192    
_________________________________________________________________
dropout (Dropout)            (None, 100, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 60)                7

In [18]:
model.fit(X, y, batch_size=256, epochs=20)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 132434 samples
Epoch 1/20
132434/132434 [==============================] - 200s 2ms/sample - loss: 3.1079
Epoch 2/20
132434/132434 [==============================] - 194s 1ms/sample - loss: 2.9432
Epoch 3/20
132434/132434 [==============================] - 198s 1ms/sample - loss: 2.7535
Epoch 4/20
132434/132434 [==============================] - 206s 2ms/sample - loss: 2.6728
Epoch 5/20
132434/132434 [==============================] - 208s 2ms/sample - loss: 2.6211
Epoch 6/20
132434/132434 [==============================] - 211s 2ms/sample - loss: 2.5786
Epoch 7/20
132434/132434 [==============================] - 212s 2ms/sample - loss: 2.5372
Epoch 8/20
132434/132434 [==============================] - 212s 2ms/sample - loss: 2.5004
Epoch 9/20
132434/132434 [==============================] - 212s 2ms/sample - loss: 2.4625
Epoch 10/20
132434/132434 [==============================] - 21

In [19]:
start = np.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([indices_char[value] for value in pattern]), "\"")

Random Seed:
" e unnatural; and as for the
notion of his preferring wine to brandy, i entirely disbelieved it. the "


In [21]:
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(chars))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = indices_char[index]
    seq_in = [indices_char[value] for value in pattern]

    sys.stdout.write(result)
    #print(result)  this will print character line by line
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

 the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh the shoueh the 
she sooe of the shoueh the shoueh the shoueh t